In [30]:
import cv2
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'cv2'

In [28]:
import sys
print(sys.executable)


/usr/local/opt/python@3.10/bin/python3.10


In [51]:
import sys
print(sys.executable)

import cv2
print("OpenCV version:", cv2.__version__)


/usr/local/opt/python@3.10/bin/python3.10


ModuleNotFoundError: No module named 'cv2'

In [ ]:
model = YOLO('yolov8s.pt')

In [ ]:
image_path = 'https://ultralytics.com/images/bus.jpg'
results = model(image_path, conf=0.5)
results[0].show()
